# Check PSF correction in A360 field

Contact author: Céline Combet; with inputs from many (Anthony, Ian, Miranda, Shenming, Anja, etc.)\
LSST Science Piplines version: Weekly 2025_09\
Container Size: large

This notebook aims at check the PSF behaviour and correction in A360 field, that we use to perform the WL analysis. The main steps are

- Loading the relevant stars from the object catalogs (all tracts and patches needed) using the butler
- Checking out the size of the PSF accross the field
- Computing the ellipticities of stars and corresponding PSF model and make the whisker plots to check out the residuals.
- Compute the radial profile of the tangential residuals (this uses CLMM)

All is done in the `i` band.

NB: Check out the [PSF DP0.2 tutorial](https://github.com/lsst/tutorial-notebooks/blob/main/DP0.2/12b_PSF_Science_Demo.ipynb) for more PSF diagnostics 

In [ ]:
# general python packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from lsst.daf.butler import Butler
import lsst.geom as geom
import lsst.afw.geom as afwGeom

In [ ]:
# repo = '/repo/main'
# collection = 'LSSTComCam/runs/DRP/DP1/w_2025_08/DM-49029'
# collection = 'LSSTComCam/runs/DRP/DP1/w_2025_09/DM-49235'

repo = '/repo/dp1'
collection = 'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260'

butler = Butler(repo, collections=collection)
skymap = butler.get('skyMap', skymap='lsst_cells_v1')

In [ ]:
version_str = collection.split('/')
version = version_str[-2:][0]+'_'+version_str[-2:][1]
version

## Load the relevant catalogs
For PSF studies, we need to look at stars
### Find all tracts/patches to load

In [ ]:
# Position of the BCG for A360
ra_bcg = 37.862
dec_bcg = 6.98

# Looking for all patches in delta deg region around it
delta = 0.5
center = geom.SpherePoint(ra_bcg, dec_bcg, geom.degrees)
ra_min, ra_max = ra_bcg - delta, ra_bcg + delta
dec_min, dec_max = dec_bcg - delta, dec_bcg + delta

ra_range = (ra_min, ra_max)
dec_range = (dec_min, dec_max)
radec = [geom.SpherePoint(ra_range[0], dec_range[0], geom.degrees),
         geom.SpherePoint(ra_range[0], dec_range[1], geom.degrees),
         geom.SpherePoint(ra_range[1], dec_range[0], geom.degrees),
         geom.SpherePoint(ra_range[1], dec_range[1], geom.degrees)]

tracts_and_patches = skymap.findTractPatchList(radec)

tp_dict = {}
for tract_num in np.arange(len(tracts_and_patches)):
    tract_info = tracts_and_patches[tract_num][0]
    tract_idx = tract_info.getId()
    # All the patches around the cluster
    patches = []
    for i,patch in enumerate(tracts_and_patches[tract_num][1]):
        patch_info = tracts_and_patches[tract_num][1][i]
        patch_idx = patch_info.sequential_index
        patches.append(patch_idx)
    tp_dict.update({tract_idx:patches})
#tp_dict

### Load quantities with the cuts needed to get PSF stars, etc. 

In [ ]:
# Get the object catlaog of these patches
if 'v29' in version:
    datasetType = 'object_patch'
else:
    datasetType = 'objectTable'

merged_cat_used = pd.DataFrame() # to store the catalog of stars used by PIFF for the PSF modeling
merged_cat_reserved = pd.DataFrame() # to store the catalog of stars marked as "reserved", i.e. not used to build the PIFF PSF model 
merged_cat_all = pd.DataFrame() # to store all star-like objects, to have more locations to check the PSF model.

for tract in list(tp_dict.keys()):
    print(f'Loading objects from tract {tract}, patches:{tp_dict[tract]}')
    for patch in tp_dict[tract]:
#        print(patch)
        dataId = {'tract': tract, 'patch' : patch ,'skymap':'lsst_cells_v1'}
        obj_cat = butler.get(datasetType, dataId=dataId)
        if datasetType == 'object_patch': # new naming convention, and obj_cat is now an astropy table. 
            obj_cat = obj_cat.to_pandas() # convert to pandas to leave the rest of the code unchanged

        # Stars used for the PSF modeling
        filt = obj_cat['detect_isPrimary'] == True
        filt &= obj_cat['refExtendedness'] == 0.0 # keep stars only
        filt &= obj_cat['i_calib_psf_used'] == True # that were used to build the psf model
        filt &= obj_cat['i_pixelFlags_inexact_psfCenter'] == False # To avoid objects with discontinuous PSF (due to edges)
        filt &= obj_cat['i_calibFlux'] > 360 # nJy, be bright
        merged_cat_used = pd.concat([merged_cat_used, obj_cat[filt]], ignore_index=True)

        # Stars "reserved" to check the PSF modeling
        filt = obj_cat['detect_isPrimary'] == True
        filt &= obj_cat['refExtendedness'] == 0.0
        filt &= obj_cat['i_calib_psf_reserved'] == True # not used for the psf model
        filt &= obj_cat['i_pixelFlags_inexact_psfCenter']==False
        merged_cat_reserved = pd.concat([merged_cat_reserved, obj_cat[filt]], ignore_index=True)

        # All extended objects (to have more locations where to look at the PSF size)
        filt = obj_cat['detect_isPrimary']==True
        filt &= obj_cat['refExtendedness'] == 1.0
        merged_cat_all = pd.concat([merged_cat_all, obj_cat[filt]], ignore_index=True)

## Check out the location of the PSF stars, in (ra, dec) and (x,y) coordinates, colored by track number

The BCG and 1 deg field around it are highlighted in the (ra,dec) plot

In [ ]:
from matplotlib.patches import Circle

circle1 = Circle((ra_bcg, dec_bcg), 0.5, color='black', fill=False, linewidth=0.5)

color = ['red', 'blue','green','magenta']

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
for i,tract in enumerate(list(tp_dict.keys())):
    filt = merged_cat_used['tract'] == tract
    ax[0].scatter(merged_cat_used[filt]['coord_ra'], merged_cat_used[filt]['coord_dec'], 
                  c=color[i],  marker='.', s=2, label=f'tract = {tract}')    
    ax[0].set_xlabel('ra [deg]')
    ax[0].set_ylabel('dec [deg]')
    ax[0].add_patch(circle1)

for i,tract in enumerate(list(tp_dict.keys())):
    filt = merged_cat_used['tract'] == tract
    ax[1].scatter(merged_cat_used[filt]['i_centroid_x'], merged_cat_used[filt]['i_centroid_y'],  marker='.', s=2, c=color[i])
ax[1].set_xlabel('i_centroid_x')
ax[1].set_ylabel('i_centroid_y')
fig.tight_layout()

ax[0].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='black')
ax[0].invert_xaxis() # to have ra increase to the left/east

fig.legend(loc=9, markerscale=10)

In (ra,dec), the stars cover the field and we can see which tract contribute to which area. In (x,y), we see a clear gap between the tracts as each tract has its own x, y coordinate system, (and some of these tracts do not have any visits covering some parts of them). Nontheless, looking at the patterns in the tracts, we see that the (x,y) grid is align with (ra,dec).

## PSF size variation across the field

The `i_i{xx,xy,yy}PSF` quantities are the second moment of the PSF model for each object location in the catalog. The trace radius (PSF size) of the PSF is defined as
$r_t = \sqrt{(I_{xx} + I_{yy}/2)}$

We look at the size of the PSF:
- at the location of `used` stars (`merged_cat_used` catalog)
- at the location of all extended objects (`merged_cat_all` catalog), to have a better coverage of the field and visualize PSF discontinutities, etc.


In [ ]:
size = np.sqrt((merged_cat_used['i_ixxPSF'] + merged_cat_used['i_iyyPSF']) / 2)
size_all = np.sqrt((merged_cat_all['i_ixxPSF'] + merged_cat_all['i_iyyPSF']) / 2) # at all extended objects locations (not stars)

In [ ]:
from matplotlib.patches import Circle

ra, dec =  merged_cat_used['coord_ra'], merged_cat_used['coord_dec']

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,5))

scatter_plot1 = ax[0].scatter(ra, dec, c=size, s=4, cmap='viridis', marker='o')
circle1 = Circle((ra_bcg, dec_bcg), 0.5, color='orange', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')

ax[0].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='orange')
ax[0].invert_xaxis()
ax[0].add_patch(circle1)
#ax.add_patch(circle2)

scatter_plot2 = ax[1].scatter(merged_cat_all['coord_ra'], merged_cat_all['coord_dec'], 
                              c=size_all, s=1, cmap='viridis', marker='o')
ax[1].set_xlabel('ra')
ax[1].set_ylabel('dec')
ax[1].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='orange')
circle2 = Circle((ra_bcg, dec_bcg), 0.5, color='orange', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')
ax[1].invert_xaxis()
ax[1].add_patch(circle2)

plt.colorbar(scatter_plot1, ax=ax[0], label='PSF size [pixels]')
plt.colorbar(scatter_plot1, ax=ax[1], label='PSF size [pixels]')

The PSF size is varying by ~0.5 pixel across A360 field. The figure on the right showis the PSF size at more 
locations, which highlights the discontinuities in the PSF modeling when close to edges. Also allows us to see the various orientation of visits used to build the coadd.

## Whisker plots - PSF ellipticity and PSF correction.

The ellipticity components $e_1$, $e_2$ are computed from moments as:

$e_1 = (I_{xx} - I_{yy}) / (I_{xx} + I_{yy})$

$e_2 = 2I_{xy} / (I_{xx} + I_{yy})$

The from this, the amplitude and orientation of the ellipse (angle of the ellipse major axis with respect to the (x,y) coordinate frame) are given by

$e = \sqrt{e_1^2 + e_2^2}$

$\theta = 0.5 \times \arctan (e_2/e_1)$

In [ ]:
def get_psf_ellip(catalog):
    psf_mxx = catalog['i_ixxPSF']
    psf_myy = catalog['i_iyyPSF']
    psf_mxy = catalog['i_ixyPSF']
    return (psf_mxx - psf_myy) / (psf_mxx + psf_myy), 2.* psf_mxy / (psf_mxx + psf_myy)


def get_star_ellip(catalog):
    star_mxx = catalog['i_ixx']
    star_myy = catalog['i_iyy']
    star_mxy = catalog['i_ixy']
    return (star_mxx - star_myy) / (star_mxx + star_myy), 2. * star_mxy / (star_mxx + star_myy)

    

## Whisker plot and residuals at the locations of `used` stars

In [ ]:
# For the PSF model, at the location of `used` stars
e1_psf_used, e2_psf_used = get_psf_ellip(merged_cat_used)
e_psf_used = np.sqrt(e1_psf_used*e1_psf_used + e2_psf_used*e2_psf_used) # module of ellipticity
theta_psf_used = 0.5 * np.arctan(e2_psf_used/e1_psf_used) # orientation

cx_psf_used = e_psf_used * np.cos(theta_psf_used) # x-component of the vector for the whisker plot
cy_psf_used = e_psf_used * np.sin(theta_psf_used) # y-component of the vector for the whisker plot

In [ ]:
# Repeat for the `used` stars
e1_star_used, e2_star_used = get_star_ellip(merged_cat_used)
e_star_used = np.sqrt(e1_star_used*e1_star_used+e2_star_used*e2_star_used)
theta_star_used = 0.5 * np.arctan(e2_star_used/e1_star_used)

cx_star_used = e_star_used * np.cos(theta_star_used)
cy_star_used = e_star_used * np.sin(theta_star_used)

### in (x,y) coordinates

In [ ]:
x_centroid, y_centroid =  merged_cat_used['i_centroid_x'], merged_cat_used['i_centroid_y']

scale = 10000

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,5))
ax[0].quiver(x_centroid, y_centroid, scale*cx_star_used, scale*cy_star_used, angles='xy', color='black',
           scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
         label='Star ellipticity')
ax[0].set_title('Star ellipticity (from i_ixx, i_ixy, i_iyy)')
ax[0].set_xlabel('x_centroid')
ax[0].set_ylabel('y_centroid')

ax[1].quiver(x_centroid, y_centroid, scale*cx_psf_used, scale*cy_psf_used, angles='xy', color='black',
           scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
         label='PSF model ellipticity')
ax[1].set_xlabel('x_centroid')
ax[1].set_title('PSF model ellipticity (from i_ixxPSF, i_ixyPSF, i_iyyPSF)')

ax[2].quiver(x_centroid, y_centroid, scale*(cx_star_used-cx_psf_used), scale*(cy_star_used-cy_psf_used), angles='xy', color='black',
           scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0)

ax[2].set_xlabel('x_centroid')
ax[2].set_title('Star - PSF ellipticity residuals')

ref_norm = scale*0.1  # Define reference vector norm
ref_x, ref_y = 3000, 5000  # Position to place reference vector
ax[0].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')
ax[1].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')
ax[2].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')

ax[0].text(ref_x + ref_norm / 2, ref_y + 500, f'e=0.1', color='red', ha='center')
ax[1].text(ref_x + ref_norm / 2, ref_y + 500, f'e=0.1', color='red', ha='center')
ax[2].text(ref_x + ref_norm / 2, ref_y + 500, f'e=0.1', color='red', ha='center')

# 
fig.tight_layout()


### in (ra, dec)

To be consistent with ra increasing to the left, need to switch the sign of the `cx` component for the plots.

In [ ]:
x_centroid, y_centroid =  merged_cat_used['coord_ra'], merged_cat_used['coord_dec']

scale = 0.6


circle0 = Circle((ra_bcg, dec_bcg), 0.5, color='darkviolet', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')
circle1 = Circle((ra_bcg, dec_bcg), 0.5, color='darkviolet', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')
circle2 = Circle((ra_bcg, dec_bcg), 0.5, color='darkviolet', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')


fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,5))
ax[0].quiver(x_centroid, y_centroid, -scale*cx_star_used, scale*cy_star_used, angles='xy', color='black',
           scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
         label='Star ellipticity')
ax[0].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='darkviolet')
ax[0].add_patch(circle0)

ax[0].invert_xaxis()
ax[0].set_title('Star ellipticity (from i_ixx, i_ixy, i_iyy)')
ax[0].set_xlabel('ra')
ax[0].set_ylabel('dec')
ax[0].set_xlim([38.6, 37.2])
ax[0].set_ylim([6.25, 7.6])

ax[1].quiver(x_centroid, y_centroid, -scale*cx_psf_used, scale*cy_psf_used, angles='xy', color='black',
           scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
         label='PSF model ellipticity')
ax[1].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='darkviolet')
ax[1].add_patch(circle1)

ax[1].invert_xaxis()
ax[1].set_xlabel('ra')
ax[1].set_title('PSF model ellipticity (from i_ixxPSF, i_ixyPSF, i_iyyPSF)')
ax[1].set_xlim([38.6, 37.2])
ax[1].set_ylim([6.25, 7.6])

ax[2].quiver(x_centroid, y_centroid, -scale*(cx_star_used-cx_psf_used), scale*(cy_star_used-cy_psf_used), angles='xy', color='black',
           scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0)
ax[2].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='darkviolet')
ax[2].add_patch(circle2)

ax[2].invert_xaxis()
ax[2].set_xlabel('ra')
ax[2].set_title('Star - PSF ellipticity residuals')
ax[2].set_xlim([38.6, 37.2])
ax[2].set_ylim([6.25, 7.6])

ref_norm = scale*0.1  # Define reference vector norm
ref_x, ref_y = 38.4, 6.27  # Position to place reference vector
ax[0].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')
ax[1].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')
ax[2].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')

ax[0].text(ref_x + ref_norm / 2, ref_y + 0.02, f'e=0.1', color='red', ha='center')
ax[1].text(ref_x + ref_norm / 2, ref_y + 0.02, f'e=0.1', color='red', ha='center')
ax[2].text(ref_x + ref_norm / 2, ref_y + 0.02, f'e=0.1', color='red', ha='center')


# ax[1].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='orange')
# ax[1].add_patch(circle1)
# ax[2].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='orange')
# ax[2].add_patch(circle1)
# 
fig.tight_layout()


The violet circle is a 1 deg field around the BCG.

## Whisker plot and residuals at the locations of `reserved` stars (that haven't been used by PIFF)

Now we repeat the same thing with the `reserved` stars, that were not used to buid the PSF model. NB: there are far less reserved stars than PSF stars.

In [ ]:
x_centroid, y_centroid =  merged_cat_reserved['coord_ra'], merged_cat_reserved['coord_dec']

scale = 0.6


circle0 = Circle((ra_bcg, dec_bcg), 0.5, color='darkviolet', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')
circle1 = Circle((ra_bcg, dec_bcg), 0.5, color='darkviolet', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')
circle2 = Circle((ra_bcg, dec_bcg), 0.5, color='darkviolet', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')


fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,5))
ax[0].quiver(x_centroid, y_centroid, -scale*cx_star_reserved, scale*cy_star_reserved, angles='xy', color='black',
           scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
         label='Star ellipticity')
ax[0].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='darkviolet', label='cluster 0.5 deg field')
ax[0].add_patch(circle0)

ax[0].invert_xaxis()
ax[0].set_title('Star ellipticity (from i_ixx, i_ixy, i_iyy)')
ax[0].set_xlabel('ra')
ax[0].set_ylabel('dec')
ax[0].set_xlim([38.6, 37.2])
ax[0].set_ylim([6.25, 7.6])

ax[1].quiver(x_centroid, y_centroid, -scale*cx_psf_reserved, scale*cy_psf_reserved, angles='xy', color='black',
           scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
         label='PSF model ellipticity')
ax[1].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='darkviolet')
ax[1].add_patch(circle1)

ax[1].invert_xaxis()
ax[1].set_xlabel('ra')
ax[1].set_title('PSF model ellipticity (from i_ixxPSF, i_ixyPSF, i_iyyPSF)')
ax[1].set_xlim([38.6, 37.2])
ax[1].set_ylim([6.25, 7.6])

ax[2].quiver(x_centroid, y_centroid, -scale*(cx_star_reserved-cx_psf_reserved), scale*(cy_star_reserved-cy_psf_reserved), angles='xy', color='black',
           scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0)
ax[2].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='darkviolet')
ax[2].add_patch(circle2)

ax[2].invert_xaxis()
ax[2].set_xlabel('ra')
ax[2].set_title('Star - PSF ellipticity residuals')
ax[2].set_xlim([38.6, 37.2])
ax[2].set_ylim([6.25, 7.6])

ref_norm = scale*0.1  # Define reference vector norm
ref_x, ref_y = 38.4, 6.27  # Position to place reference vector
ax[0].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')
ax[1].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')
ax[2].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')

ax[0].text(ref_x + ref_norm / 2, ref_y + 0.02, f'e=0.1', color='red', ha='center')
ax[1].text(ref_x + ref_norm / 2, ref_y + 0.02, f'e=0.1', color='red', ha='center')
ax[2].text(ref_x + ref_norm / 2, ref_y + 0.02, f'e=0.1', color='red', ha='center')

# ax[1].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='orange')
# ax[1].add_patch(circle1)
# ax[2].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='orange')
# ax[2].add_patch(circle1)
# 
fig.tight_layout()


## Histogram of the e1, e2 residuals in A360 field, for `used` and `reserved` stars

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))

ax[0].hist((e1_star_used-e1_psf_used), bins=30, range=[-0.04, 0.04], density=True, alpha=0.2, label='used');
ax[0].hist((e1_star_reserved-e1_psf_reserved), bins=30, range=[-0.04, 0.04], density=True, alpha=0.2, label='reserved')
ax[0].set_xlabel('Image e1 - PSF e1')
ax[0].set_xlabel(r'$\delta e_1 = $ Star e1 - PSF e1')
ax[0].legend()

ax[1].hist((e2_star_used-e2_psf_used), bins=30, range=[-0.04, 0.04], density=True, alpha=0.2, label='used');
ax[1].hist((e2_star_reserved-e2_psf_reserved), bins=30, range=[-0.04, 0.04], density=True, alpha=0.2, label='reserved')
ax[1].set_xlabel(r'$\delta e_2 = $ Star e2 - PSF e2')
ax[1].legend()

## Radial profile of the tangential residuals (uses CLMM)

Another diagnostic test of the PSF correction consists in computing the tangential residuals from $\delta e_1$ and $\delta e_2$, and plot the corresponding radial profiles from the cluster BCG, for both `used` and `reserved` stars.

In [ ]:
import clmm
from clmm import GalaxyCluster, GCData, Cosmology
from clmm import Cosmology, utils

In [ ]:
cosmo = clmm.Cosmology(H0=70.0, Omega_dm0=0.3 - 0.045, Omega_b0=0.045, Omega_k0=0.0)

### `used` stars without and with the PSF correction

In [ ]:
galcat = GCData()
galcat['ra'] = merged_cat_used['coord_ra']
galcat['dec'] = merged_cat_used['coord_dec']
galcat['e1'] = e1_star_used - e1_psf_used
galcat['e2'] = e2_star_used - e2_psf_used
#galcat['e_err'] = e_err[to_use]/2.  # factor 2 to account for conversion between e and g

galcat['z'] = np.zeros(len(galcat['ra'])) # CLMM needs a redshift column for the source, even if not used

cluster_id = "Abell 360"
gc_object1 = clmm.GalaxyCluster(cluster_id, ra_bcg, dec_bcg, 0.22, galcat, 
                                coordinate_system='euclidean')


gc_object1.compute_tangential_and_cross_components(add=True);

#print(len(gc_object1.galcat))

bins_mpc = clmm.make_bins(0.4,7,nbins=7, method='evenlog10width')
gc_object1.make_radial_profile(bins=bins_mpc, bin_units='Mpc', add=True, cosmo=cosmo, overwrite=True, 
                               use_weights=False, error_model='ste');

#print(gc_object1.profile)

galcat = GCData()
galcat['ra'] = merged_cat_used['coord_ra']
galcat['dec'] = merged_cat_used['coord_dec']
galcat['e1'] = e1_star_used# - e1_psf_used
galcat['e2'] = e2_star_used# - e2_psf_used
#galcat['e_err'] = e_err[to_use]/2.  # factor 2 to account for conversion between e and g
galcat['z'] = np.zeros(len(galcat['ra'])) # CLMM needs a redshift column for the source, even if not used

gc_object2 = clmm.GalaxyCluster(cluster_id, ra_bcg, dec_bcg, 0.22, galcat, 
                                coordinate_system='euclidean')


gc_object2.compute_tangential_and_cross_components(add=True);

#print(len(gc_object2.galcat))

bins_mpc = clmm.make_bins(0.4,7,nbins=7, method='evenlog10width')
gc_object2.make_radial_profile(bins=bins_mpc, bin_units='Mpc', add=True, cosmo=cosmo, overwrite=True, 
                               use_weights=False, error_model='ste');

#print(gc_object2.profile)


plt.errorbar(gc_object2.profile['radius']*1.02, gc_object2.profile['gt'], gc_object2.profile['gt_err'], 
             ls='', marker='.', label=r"'used' stars - $e_t$, no PSF correction")
plt.errorbar(gc_object1.profile['radius'], gc_object1.profile['gt'], gc_object1.profile['gt_err'], 
             ls='', marker='.', label=r'residuals, $\delta e_t$')

plt.xscale('log')
plt.axhline(0.0, color='k', ls=':')
#plt.ylim([-0.01,0.01])
plt.ylim([-0.06,0.05])
plt.xlim([0.3,7])
#plt.yscale('log')
plt.xlabel('R [Mpc]')
plt.ylabel('tangential component')
plt.legend(loc=1)
plt.savefig(f"profile_{version}.png")

The PSF model appears to nicely correct the tangential profile. The level of the residuals is typically an order of magnitude lower than the lensing signal of the cluster.

### Radial profile of the tangential residuals, for `used` and `reserved` stars

Now, we check the residuals on `reserved` stars, that have not been used to build the PSF model. There are ~10 times less `reserved` stars than `used` stars. For this reason, the first bins of the `reserved` star tangential residual profile may be empty.

In [ ]:
################ Reserved stars #############
galcat = GCData()
galcat['ra'] = merged_cat_reserved['coord_ra']
galcat['dec'] = merged_cat_reserved['coord_dec']
galcat['e1'] = e1_star_reserved - e1_psf_reserved # delta e1
galcat['e2'] = e2_star_reserved - e2_psf_reserved # delta e1

galcat['z'] = np.zeros(len(galcat['ra'])) # CLMM needs a redshift column for the source, even if not used

cluster_id = "Abell 360"
gc_object1 = clmm.GalaxyCluster(cluster_id, ra_bcg, dec_bcg, 0.22, galcat, 
                                coordinate_system='euclidean')

gc_object1.compute_tangential_and_cross_components(add=True);

#print(len(gc_object1.galcat))

bins_mpc = clmm.make_bins(0.4,7,nbins=7, method='evenlog10width')
gc_object1.make_radial_profile(bins=bins_mpc, bin_units='Mpc', add=True, cosmo=cosmo, overwrite=True, 
                               use_weights=False, error_model='ste');

#print(gc_object1.profile)

################ Used stars #############

galcat = GCData()
galcat['ra'] = merged_cat_used['coord_ra']
galcat['dec'] = merged_cat_used['coord_dec']
galcat['e1'] = e1_star_used - e1_psf_used
galcat['e2'] = e2_star_used - e2_psf_used
#galcat['e_err'] = e_err[to_use]/2.  # factor 2 to account for conversion between e and g
galcat['z'] = np.zeros(len(galcat['ra'])) # CLMM needs a redshift column for the source, even if not used

gc_object2 = clmm.GalaxyCluster(cluster_id, ra_bcg, dec_bcg, 0.22, galcat, 
                                coordinate_system='euclidean')


gc_object2.compute_tangential_and_cross_components(add=True);

#print(len(gc_object2.galcat))

gc_object2.make_radial_profile(bins=bins_mpc, bin_units='Mpc', add=True, cosmo=cosmo, overwrite=True, 
                               use_weights=False, error_model='ste');

#print(gc_object2.profile)

print(f'Number of used stars = {len(gc_object2.galcat)}')
print(f'Number of reserved stars = {len(gc_object1.galcat)}')

plt.errorbar(gc_object1.profile['radius'], gc_object1.profile['gt'], gc_object1.profile['gt_err'], 
             ls='', marker='.', label="'reserved' stars - residuals")
plt.errorbar(gc_object2.profile['radius'], gc_object2.profile['gt'], gc_object2.profile['gt_err'], 
             ls='', marker='.', label="'used' stars - residuals")

plt.xscale('log')
plt.axhline(0.0, color='k', ls=':')
plt.ylim([-0.007,0.007])
#plt.ylim([-0.06,0.05])
plt.xlim([0.3,7])
#plt.yscale('log')
plt.xlabel('R [Mpc]')
plt.ylabel('e_t')
plt.legend(loc=1)
plt.savefig(f"profile_{version}.png")